<a href="https://colab.research.google.com/github/rubuntu/uaa-417-sistemas-de-gestion-de-bases-de-datos-avanzados/blob/main/pgvector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# Instalar postgresql
!if [ -f /etc/apt/sources.list.d/pitti-postgresql-$(lsb_release --codename --short).list ]; then rm /etc/apt/sources.list.d/pitti-postgresql-$(lsb_release --codename --short).list; fi
!echo "deb http://apt.postgresql.org/pub/repos/apt/ $(lsb_release --codename --short)-pgdg main" > /etc/apt/sources.list.d/pgdg.list
!wget --quiet -O - http://apt.postgresql.org/pub/repos/apt/ACCC4CF8.asc | apt-key add -
!apt-get update
!apt-get install pgdg-keyring

!apt-get install postgresql-17-pgvector postgresql-contrib-17

In [2]:
# Obtener datos
!git clone https://github.com/rubuntu/datasets.git

Cloning into 'datasets'...
remote: Enumerating objects: 1296, done.
remote: Counting objects: 100% (1296/1296), done.
remote: Compressing objects: 100% (297/297), done.
remote: Total 1296 (delta 994), reused 1289 (delta 990), pack-reused 0 (from 0)
Receiving objects: 100% (1296/1296), 20.52 MiB | 24.63 MiB/s, done.
Resolving deltas: 100% (994/994), done.


In [3]:
# Iniciar postgresql
!service postgresql start
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres psql -c "CREATE DATABASE ragdb;"

# Crear la extensión pgvector
!sudo -u postgres psql -d ragdb -c "CREATE EXTENSION IF NOT EXISTS vector;"
#!sudo -u postgres psql -d ragdb -c "\\dx"

 * Starting PostgreSQL 17 database server
   ...done.
ALTER ROLE
CREATE DATABASE
CREATE EXTENSION


In [4]:
%%capture
# Instalar bibliotecas necesarias
!pip install openai psycopg2-binary sentence-transformers

In [5]:
import warnings; warnings.filterwarnings('ignore')
import os
import openai
import psycopg2
from psycopg2.extensions import register_adapter, AsIs
from psycopg2.extras import execute_values
from sentence_transformers import SentenceTransformer

In [6]:
# Configurar modelo para embeddings
#model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Configuración de la conexión a PostgreSQL
DB_NAME = "ragdb"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [7]:
def connect_to_db():
    """Establecer conexión con la base de datos PostgreSQL."""
    return psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )

def create_table():
    """Crear tabla para almacenar los embeddings."""
    query = """
    CREATE TABLE IF NOT EXISTS markdown_embeddings (
        id SERIAL PRIMARY KEY,
        file_path TEXT NOT NULL,
        content TEXT NOT NULL,
        embedding VECTOR(512)
    );
    """
    with connect_to_db() as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

def process_markdown_files(root_dir):
    """Leer archivos Markdown y generar embeddings."""
    data_to_insert = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.md'):
                file_path = os.path.join(dirpath, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    embedding = model.encode(content).tolist()
                    data_to_insert.append((file_path, content, embedding))
    return data_to_insert

def insert_embeddings(data):
    """Insertar embeddings en la base de datos."""
    query = """
    INSERT INTO markdown_embeddings (file_path, content, embedding)
    VALUES %s
    """
    with connect_to_db() as conn:
        with conn.cursor() as cur:
            execute_values(cur, query, data, template=None)
            conn.commit()

In [8]:
import warnings; warnings.filterwarnings('ignore')

# Crear tabla si no existe
create_table()

# Procesar archivos Markdown
root_directory = "datasets/www.indufar.com.py/"
embeddings_data = process_markdown_files(root_directory)

# Insertar datos en la base de datos
insert_embeddings(embeddings_data)
print(f"Procesados e insertados {len(embeddings_data)} archivos Markdown en la base de datos.")

Procesados e insertados 123 archivos Markdown en la base de datos.


In [9]:
# Generar el embedding de la pregunta
question = "¿Qué productos tiene Indufar para la fiebre?"
question_embedding = model.encode(question).tolist()

In [10]:
# Crear la extensión si no existe
def ensure_pgvector_extension():
    with psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    ) as conn:
        with conn.cursor() as cur:
            cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
            conn.commit()

# Buscar documentos más cercanos a la pregunta
def find_similar_documents(embedding):
    query = """
        SELECT file_path, content, embedding <-> vector(%s) AS distance
        FROM markdown_embeddings
        ORDER BY distance ASC
        LIMIT 5;
    """
    # Convertir embedding a formato compatible con PostgreSQL
    embedding_str = f"[{','.join(map(str, embedding))}]"

    with psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    ) as conn:
        with conn.cursor() as cur:
            cur.execute(query, (embedding_str,))
            results = cur.fetchall()
    return results

In [11]:
# Asegurar la extensión y buscar documentos
ensure_pgvector_extension()  # Crear la extensión si no existe

# Buscar documentos similares
results = find_similar_documents(question_embedding)

# Mostrar resultados
for result in results:
    file_path, content, distance = result
    print(f"Archivo: {file_path}\nDistancia: {distance}\nContenido: {content[:100]}...\n")
    #print(f"Archivo: {file_path}\nDistancia: {distance}\nContenido: {content}...\n")

Archivo: datasets/www.indufar.com.py/files/shares/prospectos/400000186.md
Distancia: 0.9260957079217126
Contenido: Dermafar 
Loción
Antipruriginoso - Antialérgico
Suavizante - Refrescante
FÓRMULA
Cada 100 mL contien...

Archivo: datasets/www.indufar.com.py/files/shares/prospectos_/400000696.md
Distancia: 0.9563682283603754
Contenido: Paracetamol - Pseudoefedrina - Clorfeniramina
Antigripal - Analgésico - Antifebril
FÓRMULA
Cada sobr...

Archivo: datasets/www.indufar.com.py/files/shares/prospectos_/400000559.md
Distancia: 0.961673733659382
Contenido: Z-CAL REUMA CREMA
Diclofenac  -  Salicilato  de  Metilo
Via  de  Administración:  Tópica
Analgésico ...

Archivo: datasets/www.indufar.com.py/files/shares/prospectos_/400000818.md
Distancia: 0.9628599212270046
Contenido: Paracetamol 500 mg
Analgésico y antifebril que no irrita la mucosa gástrica.
Polvo  Efervescente
Via...

Archivo: datasets/www.indufar.com.py/files/shares/prospectos/400000364.md
Distancia: 0.9652788671461464
Contenido: Nov